# Sesta lezione. Parte A

Importeremo i file da una repositoria su github. (In qualche caso può essere scomodo pubblicare due o più file separati per dati e analisi, quando la connessione internet è assicurata la presente procedura è preferibile)

Spezziamo la URL in varie parti per chiarezza.

In [1]:
baseURL    = 'https://raw.githubusercontent.com/'
user       = 'domenicozambella/'
repository = 'BioTeIndu19/'
branch     = 'master/'

Importiamo gli stessi dati della lezione scorsa.

In [2]:
import pandas as pd
file       = 'dati/mm1.csv'
df = pd.read_csv( baseURL + user + repository + branch + file )

Plottiamo i dati della tabella

In [3]:
from bokeh.plotting import figure, show, output_notebook
param = dict(width = 700, height = 350,
             tools = 'wheel_zoom, reset,pan, box_zoom',
             x_axis_label='Concentrazione substrato',
             y_axis_label='Velocità di reazione',
            )
output_notebook()
p = figure(**param)
p.circle( 's', 'v', source=df )
show( p )

Loading BokehJS ...

<hr><br>

# Regressione non-lineare

Ricordo l'equazione di Michaelis-Menten

$$
v = \frac{V_\text{max}\cdot s}{K_\text{m} + s}
$$


Possiamo calcolare $V_\text{max}$ e $K_\text{m}$ senza prima linearizzare. Useremo una regressione non-lineare.

Ci sono moltissime possibilità. Useremo la funzione `curve_fit()` della libreria `scipy.optimize`. 

Quindi importiamo `curve_fit()` dalla libreria e definiamo la funzione (che chiamiamo `mm`) che calcola i valori teorici.

In [4]:
from scipy.optimize import curve_fit
mm = lambda s, Vmax, Km:   Vmax * s / ( Km + s)

la variabile `mm` sarà uno degli argomenti da fornire a `curve_fit()`. La prima variabile di `mm` si considera contenere i valori indipendenti. Tutte le altre variabili verranno considerati come parametri.  La funzione `curve_fit()` ricerca i valori dei parametri che minimizzano la somma dei quadrati delle distanze tra i valori teorici e quelli misurati.

Il minimo viene ricercato con prove successive. Il punto di partenza è arbitrario. È possibile inserire un punto di partenza. Non è necessario, ma nel caso esistano più minimi locali conviene scegliere un valore ragionevolmente vicino al risultato previsto. (In teoria la procedura potrebbe trovare un minimo locale diverso.)

La funzione `curve_fit()` restituisce due valori. Il primo è la tupla di parametri. Il secondo qui verr̀a ignorato (è utile per il calcolo dell'errore).

In [5]:
(Vmax, Km), _ = curve_fit(mm, df['s'], df['v'], p0=[5,5])

Plottiamo il risultato aggiungendo il grafico della curva alla figura `p` introdotta sopra.

In [6]:
from numpy import linspace       
xmax = df['s'].max()             # Estremo destro dei valori della x del grafico
s = linspace( 0, xmax, 50  )     # Un array di 50 punti tra xmin e xmax
v = mm(s, Vmax, Km)              # I valori della curva interpolante
p.title.text = 'Titolo'
p.line(s, v,                     # Aggiungiamo la curva al grafico
       legend='Vmax = {:.2f}, Km = {:.2f} (Non linear regression)'.format(Vmax, Km))
p.legend.location = 'bottom_right'
show( p )

Per curiosità potremmo confrontare il risultato con quello ottenuto dalla linearizzazione di Lineweaver-Burk.

In [7]:
from scipy.stats import linregress
df['1/s'] = 1 / df['s']
df['1/v'] = 1 / df['v']
slope, intercept, _, _, _ = linregress( df['1/s'], df['1/v'] )
Vmax1 = 1 / intercept
Km1 = slope / intercept 
v = mm(s, Vmax1, Km1)           # I valori della curva interpolante
p.line(s, v,                    # Aggiungiamo la curva al grafico
       color='brown',
       legend='Vmax = {:.2f}, Km = {:.2f} (Lineweaver-Burk)'.format(Vmax1, Km1))
show(p)

Fine

<hr><hr><hr>

La seguente cella importa file di stile HTML (può anche essere ignorata)

In [8]:
import requests
file = 'lezioni/style/custom.css'
from IPython.core.display import HTML
html_style = requests.get( baseURL + user + repository + branch + file ).text
HTML( html_style )